In [1]:
import os
import yaml
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
from plotting import plot_input, plot_potential
from SRM import SRM

## Experiment 1
Changing the patternlength in range [10, 25, 50, 75, 100]ms 

In [ ]:
from InputGenerator import InputGenerator
input_generator = InputGenerator("hyperparameters.yml")
pattern_lengths = [
    #              ('input-data-patternlength-10ms', 0.010),
#                 #    ('input-data-patternlength-25ms', 0.025),
#                 #    ('input-data-patternlength-75ms', 0.075),
                  #  ('input-data-patternlength-100ms', 0.100),
                   ('input-data-patternlength-200ms', 0.200)
                ]
# for data in pattern_lengths:
    # input_generator.generate_input_data(input_folder=data[0], patternlength=data[1])

In [2]:
def initialize_weight(initial_weight, spike_train):
    return np.ones((spike_train.shape[0],1)) * initial_weight

def run_simulation(model, spike_train, weight, pattern_times):
    model.reset_neuron()
    weight, latency = model.run(spike_train, weight, pattern_times)
    weight = np.array([w[0] for w in weight]).reshape((-1,1))
    latency = np.array([l for l in latency])
    return model, weight, latency

In [3]:
A_pos = 0.03125
B = 0.85
tau_pos = 0.0168
tau_neg = 0.0337
A_neg = -B * A_pos

threshold = 500
tau_m = 0.010
tau_s = 0.0025
K1 = 2
K2 = 4
dt = 0.001
tref = 0.001
initial_weight = 0.475

In [9]:
def read_data(input_folder):
    times = np.load(os.path.join(input_folder, "times.npy"))
    indices = np.load(os.path.join(input_folder, "indices.npy"))
    times_pattern = np.load(os.path.join(input_folder, "times_pattern.npy"))
    indices_pattern = np.load(os.path.join(input_folder, "indices_pattern.npy"))
    position_copypaste = np.load(os.path.join(input_folder, "position_copypaste.npy"))
    # sparse_spike_train = sparse.load_npz(os.path.join(input_folder, "sparse_spike_train.npz"))
    # spike_train = sparse_spike_train.toarray()
    # pattern_times = list(np.load(os.path.join(input_folder, "pattern_times.npy")))

    with open("hyperparameters.yml", "r") as yaml_file:
        hyperparameters = yaml.safe_load(yaml_file)
    dt = hyperparameters["dt"]
    # patternlength = hyperparameters["patternlength"]

    return times, indices, times_pattern, indices_pattern, position_copypaste, dt

def plot_rate_hist(output_folder, times):
    start_time = 0.0
    end_time = 0.6
    bin_edges = np.linspace(start_time, end_time, num=int(end_time/dt/10))
    plt.figure(figsize=(10, 6))
    plt.hist(times, bins=bin_edges, color='skyblue', edgecolor='black', weights=np.ones_like(times)* 100/2000)
    plt.title('population-averaged firing rates over 10 ms time bins')
    plt.xlabel('Time (s)')
    plt.ylabel('Firing rate (Hz)')
    plt.grid(True)
    plt.savefig(os.path.join(output_folder, 'rate-hist.png'))


def plot_latency(output_folder, latency):
    plt.figure(figsize=(14, 5))
    plt.plot(latency / dt, '.', alpha=0.7)
    plt.ylabel('Postsynaptic Spike Latency (ms)')
    plt.xlabel('# discharges')
    plt.savefig(os.path.join(output_folder, "latency.png"))

def plot_weight_dist(output_folder, weight):
    active_ratio = np.sum(weight>0.7)/ len(weight) * 100
    plt.figure()
    plt.plot(np.arange(len(weight)), weight, '.k')
    plt.title(f'active neuron ration (w > 0.7): {active_ratio:.1f}%')
    plt.xlabel('Neuron number')
    plt.ylabel('Weight (arbitrary units)')
    plt.savefig(os.path.join(output_folder, "weight-dist.png"))

In [ ]:
def train_model(input_folder):
  sparse_spike_train = sparse.load_npz(os.path.join(input_folder, "sparse_spike_train.npz"))
  spike_train = sparse_spike_train.toarray()
  pattern_times = list(np.load(os.path.join(input_folder, "pattern_times.npy")))

  weight = initialize_weight(initial_weight, spike_train)
  model = SRM(threshold=threshold, tau_m=tau_m, tau_s=tau_s, K1=K1, K2=K2,
          dt=dt, tref=tref,
          A_pos=A_pos, B=B, tau_pos=tau_pos, tau_neg=tau_neg)

  model, weight, latency = run_simulation(model, spike_train, weight, pattern_times)

  return model, weight, latency

In [ ]:
input_folder = "input-data-patternlength-25ms"
model_25, weight_25, latency_25 = train_model(input_folder)

In [ ]:
input_folder = "input-data-patternlength-75ms"
model_75, weight_75, latency_75 = train_model(input_folder)

In [ ]:
input_folder = "input-data-patternlength-100ms"
model_100, weight_100, latency_100 = train_model(input_folder)

In [ ]:
input_folder = "input-data-patternlength-200ms"
model_200, weight_200, latency_200 = train_model(input_folder)

In [5]:
def run_experiment(input_folder, output_folder, model, weight, latency):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    times, indices, times_pattern, indices_pattern, position_copypaste, dt = read_data(input_folder)
    patternlength = 450/len(position_copypaste)

    params = {'start_time' : 0.0,
        'end_time' : 1.0,
        'start_index' : 0,
        'end_index' : 100,}
    plot_input(times, indices, params, times_pattern, indices_pattern, save_path=os.path.join(output_folder, 'input.png'))
    plot_rate_hist(output_folder, times)
    plot_latency(output_folder, latency)
    plot_weight_dist(output_folder, weight)

    step_size = 1
    ran = np.arange(0,15,step_size)
    for r in ran:
        plot_potential(r, r+step_size, model, position_copypaste, patternlength, dt, save_path=os.path.join(output_folder, f'potential-{r}-{r+step_size}.png'))

    step_size = 5
    ran = np.arange(15,90,step_size)
    for r in ran:
        plot_potential(r, r+step_size, model, position_copypaste, patternlength, dt, save_path=os.path.join(output_folder, f'potential-{r}-{r+step_size}.png'))

    step_size = 5
    ran = np.arange(90,420,step_size)
    for r in ran:
        plot_potential(r, r+step_size, model, position_copypaste, patternlength, dt, save_path=os.path.join(output_folder, f'potential-{r}-{r+step_size}.png'))
    
    step_size = 2
    ran = np.arange(430,450,step_size)
    for r in ran:
        plot_potential(r, r+step_size, model, position_copypaste, patternlength, dt, save_path=os.path.join(output_folder, f'potential-{r}-{r+step_size}.png'))


In [6]:
input_output = [
                # ('input-data-patternlength-10ms','patternlength-experiment-10ms'),
                # ('input-data-patternlength-25ms','patternlength-experiment-25ms', model_25, weight_25, latency_25)
                # ('input-data-patternlength-75ms','patternlength-experiment-75ms', model_75, weight_75, latency_75),
                ('input-data-patternlength-100ms','patternlength-experiment-100ms', model_100, weight_100, latency_100),
                # ('input-data-patternlength-200ms','patternlength-experiment-200ms', model_200, weight_200, latency_200),
                ]

In [1]:
for input_folder, output_folder, model, weight, latency in input_output:
    run_experiment(input_folder, output_folder, model, weight, latency)

For patternlength = 200ms, the latency at the end is 77ms.

For patternlength = 100ms in the second experiment, the latency at the end for the first spike inside the pattern is 38ms
and for the second spike is 49ms! 
